In [17]:
!pip install langchain langchain-community langchain-core

  Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\andre\\AppData\\Local\\Temp\\pip-uninstall-fj2n92xu\\numpy\\linalg\\_umath_linalg.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [18]:
!pip install pypdf

In [19]:
!pip install -qU "langchain-chroma>=0.1.2"

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\andre\\anaconda3\\Lib\\site-packages\\google\\protobuf\\~~ternal\\_api_implementation.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [20]:
!pip install protobuf==3.20.*

  Using cached protobuf-3.20.3-cp39-cp39-win_amd64.whl (904 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.2
    Uninstalling protobuf-5.29.2:
      Successfully uninstalled protobuf-5.29.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
opentelemetry-proto 1.29.0 requires protobuf<6.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.


In [1]:
#from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
# from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pypdf

In [2]:
model_local = ChatOllama(model="llama3.2")

C:\Users\andre\AppData\Local\Temp\ipykernel_2356\2362477809.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model_local = ChatOllama(model="llama3.2")


In [3]:
# loader = PyPDFLoader("../../datasets/medEncyclopedia.pdf")
# docs = loader.load()

#read doc
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents
docs = read_doc("../../datasets/book")

In [4]:
#Text Splitting
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=50,
)
doc_splits = splitter.split_documents(docs)

In [ ]:
from langchain_chroma import Chroma

ids = []

#generating ids for document
for i in range(0,len(doc_splits)):
    ids.append(str(i+1))

vector_store = Chroma(
    collection_name="medassist-chroma",
    embedding_function=embeddings.OllamaEmbeddings(model='nomic-embed-text'),
    persist_directory="./chroma_db",
)

#adding documents to vector_store
vector_store.add_documents(documents=doc_splits, ids=ids)

C:\Users\andre\AppData\Local\Temp\ipykernel_22320\2837472493.py:11: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_function=embeddings.OllamaEmbeddings(model='nomic-embed-text'),


In [7]:
#retriever = vector_store.as_retriever()

In [5]:
import chromadb
from chromadb.utils import embedding_functions
default_ef = embedding_functions.DefaultEmbeddingFunction()
client = chromadb.PersistentClient(path='./chroma_db')
collection = client.get_collection(name="langchain",embedding_function=default_ef)
# db3 = Chroma(persist_directory="./chroma_db", embedding_function=embeddings.OllamaEmbeddings(model='nomic-embed-text'))

In [ ]:
# collection.similarity_search("what is effusion")
collection.query("effusion")

In [16]:
from langchain_chroma import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = Chroma(persist_directory="./chroma_db", embedding_function=default_ef)

C:\Users\andre\AppData\Local\Temp\ipykernel_13480\3989886796.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


Error importing huggingface_hub.file_download: 'Version'


KeyError: 'Version'

In [15]:
vectordb.similarity_search("effusion")

AttributeError: 'ONNXMiniLM_L6_V2' object has no attribute 'embed_query'

In [17]:
print("\n########\nAfter RAG\n")
after_rag_template = """You are a chatbot answering medical questions strictly related to chest diseases. Do not provide any medical consultations. Answer the question based only on the following context:
{context}
Question: {question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | after_rag_prompt
    | model_local
    | StrOutputParser()
)

print(after_rag_chain.invoke("what is the source of your data?"))



########
After RAG

My data comes from reputable sources in the field of cardiology and pulmonology, including:

1. Up-to-date medical textbooks such as "Harrison's Principles of Internal Medicine" and "Heart Disease: A Clinical Approach".
2. Peer-reviewed journals like The New England Journal of Medicine, Journal of the American College of Cardiology, and Chest.
3. National health organizations, including the American Heart Association (AHA) and the American Lung Association (ALA).
4. Government health resources, such as the Centers for Disease Control and Prevention (CDC) and the National Institutes of Health (NIH).

Please note that my responses are based on general information and not specific to individual cases. If you have a medical question or concern, consult a qualified healthcare professional.


In [28]:
doc_splits

[Document(metadata={'source': '..\\..\\datasets\\book\\medEncyclopedia.pdf', 'page': 1}, page_content='The GALE\nENCYCLOPEDIA of\nMEDICINE\nTHIRD EDITION'),
 Document(metadata={'source': '..\\..\\datasets\\book\\medEncyclopedia.pdf', 'page': 2}, page_content='The GALE\nENCYCLOPEDIA of\nMEDICINE\nTHIRD EDITION\nVOLUME\n\x81\n1\nA-B\nJACQUELINE L. LONGE, PROJECT EDITOR'),
 Document(metadata={'source': '..\\..\\datasets\\book\\medEncyclopedia.pdf', 'page': 3}, page_content='The GALE\nENCYCLOPEDIA of\nMEDICINE\nTHIRD EDITION\nVOLUME\n\x81\n2\nC-F\nJACQUELINE L. LONGE, PROJECT EDITOR'),
 Document(metadata={'source': '..\\..\\datasets\\book\\medEncyclopedia.pdf', 'page': 4}, page_content='The GALE\nENCYCLOPEDIA of\nMEDICINE\nTHIRD EDITION\nVOLUME\n\x81\n3\nG-M\nJACQUELINE L. LONGE, PROJECT EDITOR'),
 Document(metadata={'source': '..\\..\\datasets\\book\\medEncyclopedia.pdf', 'page': 5}, page_content='The GALE\nENCYCLOPEDIA of\nMEDICINE\nTHIRD EDITION\nVOLUME\n\x81\n4\nN-S\nJACQUELINE L. LONG

[]

[]